In [1]:
import fiona
import logging
import sys
from shapely.geometry import mapping, shape
import fiona
from collections import OrderedDict
from ws3.common import  reproject_vector_data
import os
from os import listdir
from os.path import isfile, join
from fiona.crs import CRS
from shapely.geometry import MultiPolygon


/media/data/home/eghasemi/.virtualenvs/foo/lib/python3.10/site-packages/pacal/interpolation.py:52: UserWarning: Compiled interpolation routine not available
  warnings.warn("Compiled interpolation routine not available")


Compiled sparse grid routine not available


In [2]:
dat_path = '../dat'
gdb_path = '../dat/gdb/tsa17.gdb'
shp_path = '../dat/shp/tsa17.shp'
dst_path = shp_path
src_path = gdb_path

In [3]:
logfn='clean_stand_shapefile.log'

In [4]:
dst_name = '_stands'

In [5]:
snk1_path = '%s/%s.shp' % (dst_path, dst_name) 
snk2_path = '%s/%s_error.shp' % (dst_path, dst_name) 

In [6]:
driver='ESRI Shapefile'

In [7]:
dst_epsg=None
# prop_types=None
prop_names = [u'THLB', u'AU', u'LdSpp', u'Age2015', u'Shape_Area']
prop_types = [(u'theme0', 'str:10'),
              (u'theme1', 'str:1'),
              (u'theme2', 'str:5'), 
              (u'theme3', 'str:50'), 
              (u'age', 'int:5'), 
              (u'area', 'float:10.1')]
basenames = ['tsa17']
theme0 = basenames
bn = basenames
max_records=None
snk_epsg = 3005 # ESPG:3005 corresponds to NAD83/BC Albers

In [8]:
with fiona.open(src_path, 'r') as src:
    kwds1 = src.meta.copy()
    kwds2 = src.meta.copy()
    kwds1.update(driver=driver)
    kwds2.update(driver=driver)
    if dst_epsg:
        dst_crs = from_epsg(dst_epsg)
        kwds1.update(crs=dst_crs, crs_wkt=None)
    if not prop_types:
        prop_types = [('theme0', 'str:10')] if theme0 else []
        prop_types = prop_types + [(pn.lower(), src.schema['properties'][pn]) for pn in prop_names]
    kwds1['schema']['properties'] = OrderedDict(prop_types)
    kwds2['schema']['properties'] = OrderedDict(prop_types)
    kwds1['schema']['geometry'] = 'Polygon'  
    kwds2['schema']['geometry'] = 'Polygon'  

    # Assuming snk1_path and snk2_path are the paths to your output datasets
    with fiona.open(snk1_path, 'w', **kwds1) as snk1, fiona.open(snk2_path, 'w', **kwds2) as snk2:
        n = len(src) if not max_records else max_records
        # print(n)
        for f in src[:n]:
            # print(f)
            prop_data = [('theme0', theme0)] if theme0 else []
            if prop_types:
                prop_data = prop_data + [(prop_types[i + len(prop_data)][0], f['properties'][pn])
                                         for i, pn in enumerate(prop_names)]
            else:
                prop_data = prop_data + [(pn.lower(), f['properties'][pn]) for pn in prop_names]

            # Convert list property values to strings before writing
            for key, value in prop_data:
                if isinstance(value, list):
                    prop_data[prop_data.index((key, value))] = (key, ', '.join(map(str, value)))

            try:
                g = shape(f['geometry'])
                # print(g)
                # if g.geom_type == 'Polygon':
                #     g = MultiPolygon([g]) 
                if not g.is_valid:
                    _g = g.buffer(0)
                    assert _g.is_valid
                    assert _g.geom_type == 'Polygon'
                    g = _g

                new_f = {'properties': OrderedDict(prop_data),
                         'geometry': mapping(g)
                         }
                if dst_epsg:
                    new_f = reproject(new_f, src.crs, dst_crs)
                snk1.write(new_f)
            except Exception as e:  # log exception and write uncleanable feature a separate shapefile
                logging.exception("Error cleaning feature %s:", f['id'])
                snk2.write(new_f)

with fiona.open(gdb_path) as src0, fiona.open(snk1_path) as src1, fiona.open(snk2_path) as src2:
    print('Polygons in original dataset', len(src0))
    print('Polygons in clean dataset', len(src1))
    print('Uncleanable polygons', len(src2))
reproject_vector_data(shp_path+'/_stands.shp', shp_path +'/stands.shp', snk_epsg)
_path = '%s/shp/%s.shp' % (dat_path, 'tsa17')
for f in [f for f in listdir(_path) if isfile(join(_path, f)) and f.startswith('_stands')]: 
            os.remove('%s/%s' % (_path, f))


Polygons in original dataset 45891
Polygons in clean dataset 45891
Uncleanable polygons 0


/tmp/ipykernel_543399/1878822016.py:60: FionaDeprecationWarning: This function will be removed in version 2.0. Please use CRS.from_epsg() instead.
  reproject_vector_data(shp_path+'/_stands.shp', shp_path +'/stands.shp', snk_epsg)
